## Here we demostrate why naive Bayes seems to fail for this data set.

In [54]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import sys
sys.path.insert(0,'../libs/')
import data_cleaning as dc

In [55]:
df = pd.read_csv("../data/train.csv")
df = dc.classify_colors(df)
df = dc.massage_df(df)

In [56]:
df_dog = df[df.AnimalType=='Dog'].copy(deep=True)
df_cat = df[df.AnimalType=='Cat'].copy(deep=True)

Start by fitting all the features we thought were useful.

In [57]:
dc.fix_cat_breed(df_cat)

# Selects the importand 
cat_features = ['age_numeric', 'sex', 'mixed', 'catbreed','day_of_week','month','neuter_status']
colors = ['has_Blue', 'has_Tortie',
       'has_White', 'has_Brown', 'has_Lilac', 'has_Point', 'has_Tabby',
       'has_Gray', 'has_Agouti', 'has_Black', 'has_Lynx', 'has_Orange',
       'has_Tan', 'has_Calico']

df_cat_dummies = pd.get_dummies(df_cat[cat_features + colors].dropna())
df_cat_dummies

# splits data.
rows = np.random.choice(df_cat_dummies.index.values
                ,int( round(len(df_cat_dummies)*.6) ), replace=False)
df_cat_dummies_train = df_cat_dummies.ix[rows]
df_cat_dummies_test = df_cat_dummies.drop(rows)

#### fit and test model

train_features = df_cat_dummies_train
train_outcomes = df_cat.ix[df_cat_dummies_train.index]['OutcomeType']
test_features  = df_cat_dummies_test
test_outcomes  = df_cat.ix[df_cat_dummies_test.index]['OutcomeType']

from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import accuracy_score

bnb = BernoulliNB()
bnb.fit(train_features,train_outcomes)

predictions = bnb.predict(test_features)
accuracy_score(test_outcomes,predictions)

0.73060490218124574

In [58]:
pd.Series(predictions).value_counts()

Adoption           2416
Transfer           2026
Return_to_owner       3
Euthanasia            2
dtype: int64

In [59]:
df_cat['OutcomeType'].value_counts()

Transfer           5505
Adoption           4272
Euthanasia          710
Return_to_owner     500
Died                147
Name: OutcomeType, dtype: int64

So it's missing lots of Euthanasia. It mostly just predicts transfer vs. adoption.
Let's try only neuter status:

In [60]:
dc.fix_cat_breed(df_cat)

# Selects the importand 
cat_features = ['neuter_status']

df_cat_dummies = pd.get_dummies(df_cat[cat_features + colors].dropna())
df_cat_dummies

# splits data.
rows = np.random.choice(df_cat_dummies.index.values
                ,int( round(len(df_cat_dummies)*.6) ), replace=False)
df_cat_dummies_train = df_cat_dummies.ix[rows]
df_cat_dummies_test = df_cat_dummies.drop(rows)

#### fit and test model

train_features = df_cat_dummies_train
train_outcomes = df_cat.ix[df_cat_dummies_train.index]['OutcomeType']
test_features  = df_cat_dummies_test
test_outcomes  = df_cat.ix[df_cat_dummies_test.index]['OutcomeType']

from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import accuracy_score

bnb = BernoulliNB()
bnb.fit(train_features,train_outcomes)

predictions = bnb.predict(test_features)
accuracy_score(test_outcomes,predictions)

0.72810956443646158

In [61]:
pd.Series(predictions).value_counts()

Adoption    2468
Transfer    1986
dtype: int64

Performs equally well! Seems like it's entirely based on neuter status, which works well to destinguish transfers and adoptions, likely because animals are transfered to facilities that can neuter the animal.

In [62]:
df_cat.loc[(df_cat['neuter_status']=='Intact')&(df_cat['OutcomeType']=="Adoption"),['neuter_status','OutcomeType']].count()

neuter_status    238
OutcomeType      238
dtype: int64

In [63]:
df_cat.loc[(df_cat['neuter_status']!='Intact')&(df_cat['OutcomeType']=="Adoption"),['neuter_status','OutcomeType']].count()

neuter_status    4034
OutcomeType      4034
dtype: int64

In [64]:
df_cat.loc[(df_cat['neuter_status']=='Intact')&(df_cat['OutcomeType']=="Transfer"),['neuter_status','OutcomeType']].count()

neuter_status    3234
OutcomeType      3234
dtype: int64

In [65]:
df_cat.loc[(df_cat['neuter_status']!='Intact')&(df_cat['OutcomeType']=="Transfer"),['neuter_status','OutcomeType']].count()

neuter_status    2271
OutcomeType      2271
dtype: int64

Out guess is that naive Bayes needs more data to predict things better. Neuter status is so stongly correlated with transfers that it can use this indicator, but cannot use any others.